# Qiskit Examples
In this notebook we show some examples from the [Qiskit Nature package](https://qiskit.org/documentation/nature/), using the Ingenii package to run on an Azure Quantum workspace.

### Let's get the workspace we'll submit jobs to.

First, you'll need to establish your connection to the workspace using the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/what-is-azure-cli).

If you're not in our JupyterLab space, you'll need to [install the CLI first](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli).

Then, in a terminal, run the `az login` command to take you through the authentication flow. If you don't do this now, you will be prompted later on.

In [ ]:
from ingenii_azure_quantum.qiskit import AzureQiskitAccess

# If you're in our JupyterLab space then the provider details are pre-configured in environment variables.
azure_qiskit_access = AzureQiskitAccess()

# If you're not in our JupyterLab workspace, either:
#  - set the WORKSPACE_SUBSCRIPTION_ID, WORKSPACE_RESOURCE_GROUP, WORKSPACE_LOCATION, and WORKSPACE_NAME environment variables
#  - pass these values directly to the get_workspace function, as below
#  - a combination of the two
# azure_qiskit_access.set_workspace(
#     subscription_id="<subscription ID>",
#     resource_group="<resource group name>",
#     location="<location, e.g. EastUS>",
#     name="<workspace name>",
# )

### As a first example, let's look at a protein folding solution
Adapted from the example given by Qiskit Nature: https://qiskit.org/documentation/nature/tutorials/09_Protein_Folding.html

#### Let's formulate the problem details

In [ ]:
import qiskit_nature.problems.sampling.protein_folding as pf
from qiskit.utils import algorithm_globals

# Let's set the random seed 
algorithm_globals.random_seed = 23

# Protein Main Chain
# The Protein consists of a main chain that is a linear chain of aminoacids. For the naming of different residues we use the one-letter code as defined in Ref. [3]. Further details about the naming and the type of aminoacids can also be found in [4].
# For this particular case we demonstrate the generation of the qubit operator in a neuropeptide with the main chain consisting of 7 aminoacids with letter codes APRLRFY (see also [2]).
main_chain = "APRLRFY"

# Side Chains
# Beyond the main chain of the protein there may be aminoacids attached to the residues of the main chain. Our model allows for side chains of the maximum length of one. Elongated side chains would require the introduction of additional penalty terms which are still under development. In this example we do not consider any side chains to keep the real structure of the neuropeptide
# Ingenii note: this is the default of the 'protein_folding_problem' function, so you don't need to pass this
side_chains = [""] * 7

# To ensure that all physical constraints are respected we introduce penalty functions
penalty_terms = pf.PenaltyParameters(penalty_chiral=10, penalty_back=10, penalty_1=10)

# Based on the main chain and possible side chains we define the peptide object that includes all the structural information of the modeled system.
peptide = pf.Peptide(main_chain, side_chains)

# Interaction between Aminoacids
# For the description of inter-residue contacts for proteins we use knowledge-based (statistical) potentials derived using quasi-chemical approximation. The potentials used here are introduced by Miyazawa, S. and Jernigan, R. L. in [5].
# Beyond this model we also allow for random contact maps (interactions) that provide a random interaction map. One can also introduce a custom interaction map that enhances certain configurations of the protein (e.g. alpha helix, beta sheet etc).
interaction = pf.MiyazawaJerniganInteraction()

# Based on the defined peptide, the interaction (contact map) and the penalty terms we defined for our model we define the protein folding problem that returns qubit operators
protein_folding_problem = pf.ProteinFoldingProblem(peptide, interaction, penalty_terms)

qubit_operator = protein_folding_problem.qubit_op()

print(qubit_operator)


#### Let's solve this problem using the VQE solver with CVaR expectation values

We can use the `azure_qiskit_access` object to obtain a quantum instance object that jobs can be run on.

By default, the problem solves this locally with a simulator, without submitting to the Quantum Workspace

In [ ]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import VQE
from qiskit.circuit.library import RealAmplitudes
import qiskit_nature.problems.sampling.protein_folding as pf
from qiskit.opflow import CVaRExpectation, PauliExpectation
from qiskit.utils import algorithm_globals

max_iterations = 50

# Capture the intermediate results between each Optimizer run
intermediate_results = []
def store_intermediate_result(eval_count, parameters, mean, std):
    print(f"Evaluation count: {eval_count}/{max_iterations}")
    intermediate_results.append({
        "count": eval_count,
        "parameters": parameters,
        "mean": mean,
        "standard_deviation": std,
    })

# Set the quantum instance. The default is "aer_simulator"
azure_qiskit_access.set_quantum_instance(
    shots=8192,
    seed_transpiler=algorithm_globals.random_seed,
    seed_simulator=algorithm_globals.random_seed,
)

# Initialize VQE using CVaR
cvar_exp = CVaRExpectation(0.1, PauliExpectation())
vqe = VQE(
    expectation=cvar_exp,
    optimizer=COBYLA(maxiter=50),
    ansatz=RealAmplitudes(reps=1),
    quantum_instance=azure_qiskit_access.quantum_instance,
    callback=store_intermediate_result,
)

# Get the result
results = {
    "result": vqe.compute_minimum_eigenvalue(qubit_operator),
    "intermediate_results": intermediate_results
}

for k, v in results.items():
    print(k, v)


#### As a second pass, let's solve this problem by submitting it to the Quantum Workspace
We'll need to update the `azure_qiskit_access` object with the name of the backend to use and obtain a new `quantum_instance` object.

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

# Update the quantum instance
azure_qiskit_access.set_quantum_instance(
    azure_backend_name="ionq.simulator",
    shots=8192,
    seed_transpiler=algorithm_globals.random_seed,
    seed_simulator=algorithm_globals.random_seed,
)

max_iterations = 30

intermediate_results = []
def store_intermediate_result(eval_count, parameters, mean, std):
    print(f"Evaluation count: {eval_count}/{max_iterations}")
    intermediate_results.append({
        "count": eval_count,
        "parameters": parameters,
        "mean": mean,
        "standard_deviation": std,
    })

vqe = VQE(
    expectation=cvar_exp,
    optimizer=COBYLA(maxiter=max_iterations),
    ansatz=RealAmplitudes(reps=1),
    quantum_instance=azure_qiskit_access.quantum_instance,
    callback=store_intermediate_result,
)

# Get the result
results = {
    "result": vqe.compute_minimum_eigenvalue(qubit_operator),
    "intermediate_results": intermediate_results
}

for k, v in results.items():
    print(k, v)


### References

- [1] https://en.wikipedia.org/wiki/Levinthal%27s_paradox

- [2] A.Robert, P.Barkoutsos, S.Woerner and I.Tavernelli, Resource-efficient quantum algorithm for protein folding, NPJ Quantum Information, 2021, https://doi.org/10.1038/s41534-021-00368-4

- [3] IUPAC–IUB Commission on Biochemical Nomenclature (1972). “A one-letter notation for aminoacid sequences”. Pure and Applied Chemistry. 31 (4): 641–645. doi:10.1351/pac197231040639. PMID 5080161.

- [4] https://en.wikipedia.org/wiki/Amino_acid

- [5] S. Miyazawa and R. L.Jernigan, Residue – Residue Potentials with a Favorable Contact Pair Term and an Unfavorable High Packing Density Term for Simulation and Threading, J. Mol. Biol.256, 623–644, 1996, Table 3, https://doi.org/10.1006/jmbi.1996.0114

- [6] P.Barkoutsos, G. Nannichini, A.Robert, I.Tavernelli, S.Woerner, Improving Variational Quantum Optimization using CVaR, Quantum 4, 256, 2020, https://doi.org/10.22331/q-2020-04-20-256

### As a second example, let's look at a molecule excited state solver
Adapted from the example given by Qiskit Nature: https://qiskit.org/documentation/nature/tutorials/04_excited_states_solvers.html

#### Let's formulate the problem details

In [ ]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

molecule = Molecule(
    geometry=[
        ["H", [0.0, 0.0, 0.0]],
        ["H", [0.0, 0.0, 0.735]]
    ],
    charge=0, multiplicity=1
)

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)


#### Let's solve this problem using the QEOM solver
First we'll solve this locally with a simulator, without submitting to the Quantum Workspace

In [ ]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import algorithm_globals
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

azure_qiskit_access.set_quantum_instance(
    aer_default="aer_simulator_statevector",
    shots=8192,
    seed_transpiler=algorithm_globals.random_seed,
    seed_simulator=algorithm_globals.random_seed,
)

max_iterations = 20

intermediate_results = []
def store_intermediate_result(eval_count, parameters, mean, std):
    print(f"Evaluation count: {eval_count}/{max_iterations}")
    intermediate_results.append({
        "count": eval_count,
        "parameters": parameters,
        "mean": mean,
        "standard_deviation": std,
    })

# This first part sets the ground state solver
solver = VQEUCCFactory(
    quantum_instance=azure_qiskit_access.quantum_instance,
    optimizer=COBYLA(maxiter=max_iterations),
    callback=store_intermediate_result
)
gsc = GroundStateEigensolver(QubitConverter(JordanWignerMapper()), solver)

# The qEOM algorithm is instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")

results = {
    "result": qeom_excited_states_calculation.solve(es_problem),
    "intermediate_results": intermediate_results
}

for k, v in results.items():
    print(k, v)

#### As a second pass, let's solve this problem by submitting it to the Quantum Workspace
We'll need to update the `azure_qiskit_access` object with the name of the backend to use and obtain a new `quantum_instance` object.

In [ ]:
# Update the quantum instance
azure_qiskit_access.set_quantum_instance(
    azure_backend_name="ionq.simulator",
    shots=8192,
    seed_transpiler=algorithm_globals.random_seed,
    seed_simulator=algorithm_globals.random_seed,
)

max_iterations = 20

intermediate_results = []
def store_intermediate_result(eval_count, parameters, mean, std):
    print(f"Evaluation count: {eval_count}/{max_iterations}")
    intermediate_results.append({
        "count": eval_count,
        "parameters": parameters,
        "mean": mean,
        "standard_deviation": std,
    })

# This first part sets the ground state solver
solver = VQEUCCFactory(
    quantum_instance=azure_qiskit_access.quantum_instance,
    optimizer=COBYLA(maxiter=max_iterations),
    callback=store_intermediate_result
)
gsc = GroundStateEigensolver(QubitConverter(JordanWignerMapper()), solver)

# The qEOM algorithm is instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")

results = {
    "result": qeom_excited_states_calculation.solve(es_problem),
    "intermediate_results": intermediate_results
}

for k, v in results.items():
    print(k, v)
